In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
pip install gokinjo

Note: you may need to restart the kernel to use updated packages.


In [3]:
#importing required librabies
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer,IterativeImputer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV,KFold,cross_val_score,RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score,mean_squared_error,accuracy_score
from gokinjo import knn_kfold_extract
from gokinjo import knn_extract
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [4]:
# loading csv files
df_train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
df_test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
df_train

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [5]:
# creating a copy of the datasets for further manipulation
hf_train = df_train.copy()
hf_test = df_test.copy()

In [6]:
# splitting 'Name', 'PassengerId', 'Cabin' to extract useful information
hf_train['firstname'] = hf_train['Name'].str.split(' ',expand=True)[0]
hf_train['lastname'] = hf_train['Name'].str.split(' ',expand=True)[1]
hf_train['travelgroup'] = hf_train['PassengerId'].str.split('_',expand=True)[0]
hf_train['travelgroupos'] = hf_train['PassengerId'].str.split('_',expand=True)[1]

hf_test['firstname'] = hf_test['Name'].str.split(' ',expand=True)[0]
hf_test['lastname'] = hf_test['Name'].str.split(' ',expand=True)[1]
hf_test['travelgroup'] = hf_test['PassengerId'].str.split('_',expand=True)[0]
hf_test['travelgroupos'] = hf_test['PassengerId'].str.split('_',expand=True)[1]

In [7]:
hf_train['Cabin'].fillna(value='$$',inplace=True)
hf_train[['cabin_0','cabin_1','cabin_2']] = hf_train['Cabin'].apply(lambda x: pd.Series(str(x).split('/')))
hf_train[['cabin_0','cabin_1','cabin_2']] = hf_train[['cabin_0','cabin_1','cabin_2']].replace(['','$$'],np.nan)
hf_train['cabin_1'] = hf_train['cabin_1'].astype(float)

hf_test['Cabin'].fillna(value='$$',inplace=True)
hf_test[['cabin_0','cabin_1','cabin_2']] = hf_test['Cabin'].apply(lambda x: pd.Series(str(x).split('/')))
hf_test[['cabin_0','cabin_1','cabin_2']] = hf_test[['cabin_0','cabin_1','cabin_2']].replace(['','$$'],np.nan)
hf_test['cabin_1'] = hf_test['cabin_1'].astype(float)

In [8]:
# creating 'totalbill' column by adding columns where passenger spent money
hf_train['totalbill'] = hf_train['RoomService'] + hf_train['FoodCourt'] + hf_train['Spa'] + hf_train['ShoppingMall'] + hf_train['VRDeck']

hf_test['totalbill'] = hf_test['RoomService'] + hf_test['FoodCourt'] + hf_test['Spa'] + hf_test['ShoppingMall'] + hf_test['VRDeck']

In [9]:
#creating aggregated variables based on 'cabin_0'
expend_on_each_cabin = hf_train.groupby('cabin_0').aggregate({'totalbill':'sum','PassengerId':'size','Transported':'sum'}).reset_index()
expend_on_each_cabin['avgspent'] = expend_on_each_cabin['totalbill']/expend_on_each_cabin['PassengerId']
expend_on_each_cabin['transported_percentage'] = expend_on_each_cabin['Transported']/expend_on_each_cabin['PassengerId']

hf_train = hf_train.merge(expend_on_each_cabin[['cabin_0','avgspent','transported_percentage']],how='left',on=['cabin_0'])
hf_test = hf_test.merge(expend_on_each_cabin[['cabin_0','avgspent','transported_percentage']],how='left',on=['cabin_0'])

In [10]:
#if we assume those passengers who have same lastname goes to same family 
# creating number_family_member col counting surnames
family = hf_train.groupby('lastname')['PassengerId'].count().reset_index()
family['num_fam_mem'] = family['PassengerId']
hf_train = hf_train.merge(family[['lastname','num_fam_mem']],how='left',on = ['lastname'])

family_test = hf_test.groupby('lastname')['PassengerId'].count().reset_index()
family_test['num_fam_mem'] = family_test['PassengerId']
hf_test = hf_test.merge(family_test[['lastname','num_fam_mem']],how='left',on = ['lastname'])

In [11]:
# num of people travel together in a group
group = hf_train.groupby('travelgroup')['PassengerId'].count().reset_index()
group['group'] = group['PassengerId']
hf_train= hf_train.merge(group[['travelgroup','group']],how='left',on=['travelgroup'])

group_test = hf_test.groupby('travelgroup')['PassengerId'].count().reset_index()
group_test['group'] = group_test['PassengerId']
hf_test= hf_test.merge(group_test[['travelgroup','group']],how='left',on=['travelgroup'])

In [12]:
#converting bool or str dtype to int
hf_train['VIP']=pd.to_numeric(hf_train['VIP'])
hf_train['CryoSleep']=pd.to_numeric(hf_train['CryoSleep'])
hf_train['Transported'] = hf_train['Transported'].astype(int)
hf_train[['travelgroup','travelgroupos']] = hf_train[['travelgroup','travelgroupos']].astype('int')

hf_test['VIP']=pd.to_numeric(hf_test['VIP'])
hf_test['CryoSleep']=pd.to_numeric(hf_test['CryoSleep'])
hf_test[['travelgroup','travelgroupos']] = hf_test[['travelgroup','travelgroupos']].astype('int')

In [13]:
#adding a index column for later dataset separation or sorting purpose
hf_train['index'] = range(len(hf_test),len(hf_test)+len(hf_train))
hf_test['index'] = range(len(hf_test))

In [14]:
#removing unuseful columns for later operations
hf_train = hf_train.drop(['PassengerId','firstname','lastname','Name','Cabin','VIP'],axis=1)

hf_test = hf_test.drop(['PassengerId','firstname','lastname','Name','Cabin','VIP'],axis=1)

In [15]:
#in those columnns age 13 is a threshold value for being 0. so filling some nan values with 0
def fill_nans_by_age(df, age_limit = 13):
    df['RoomService'] = np.where(df['Age'] <= age_limit, 0, df['RoomService'])
    df['FoodCourt'] = np.where(df['Age'] <= age_limit, 0, df['FoodCourt'])
    df['ShoppingMall'] = np.where(df['Age'] <= age_limit, 0, df['ShoppingMall'])
    df['Spa'] = np.where(df['Age'] <= age_limit, 0, df['Spa'])
    df['VRDeck'] = np.where(df['Age'] <= age_limit, 0, df['VRDeck'])
    return df
hf_train = fill_nans_by_age(hf_train)
hf_test = fill_nans_by_age(hf_test)

In [16]:
hf_train.corr()

,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,travelgroup,travelgroupos,cabin_1,totalbill,avgspent,transported_percentage,num_fam_mem,group,index
CryoSleep,1.000000,-0.074273,-0.253106,-0.210388,-0.212970,-0.202699,-0.196793,0.468645,-0.005879,0.064676,-0.037560,-0.385749,-0.009255,0.230689,-0.009337,0.082651,-0.006047
Age,-0.074273,1.000000,0.081597,0.132659,0.043975,0.127047,0.105447,-0.075026,-0.009439,-0.132756,-0.132255,0.189475,0.247943,0.093525,-0.133882,-0.179102,-0.009530
RoomService,-0.253106,0.081597,1.000000,-0.013658,0.057001,0.011243,-0.017582,-0.243211,0.002568,-0.024342,-0.007678,0.228603,0.028218,-0.129535,-0.029450,-0.043795,0.002671
FoodCourt,-0.210388,0.132659,-0.013658,1.000000,-0.012174,0.221847,0.229388,0.046837,-0.010860,0.023486,-0.183557,0.745826,0.353494,0.260452,-0.102376,0.031887,-0.010863
ShoppingMall,-0.212970,0.043975,0.057001,-0.012174,1.000000,0.016414,-0.004939,0.008410,0.015958,-0.040284,0.002708,0.218432,0.028043,-0.058171,-0.035127,-0.041727,0.016105
Spa,-0.202699,0.127047,0.011243,0.221847,0.016414,1.000000,0.155207,-0.219295,-0.006371,0.016871,-0.131631,0.596082,0.247401,0.167484,-0.068378,0.018564,-0.006223
VRDeck,-0.196793,0.105447,-0.017582,0.229388,-0.004939,0.155207,1.000000,-0.204962,0.015005,0.009995,-0.138942,0.572192,0.279455,0.200502,-0.075883,0.008679,0.014895
Transported,0.468645,-0.075026,-0.243211,0.046837,0.008410,-0.219295,-0.204962,1.000000,0.021491,0.066390,-0.045097,-0.197671,0.130525,0.214905,-0.033312,0.082644,0.021775
travelgroup,-0.005879,-0.009439,0.002568,-0.010860,0.015958,-0.006371,0.015005,0.021491,1.000000,0.011170,0.679723,-0.001793,0.004100,0.002985,0.004863,0.014753,0.999947
travelgroupos,0.064676,-0.132756,-0.024342,0.023486,-0.040284,0.016871,0.009995,0.066390,0.011170,1.000000,-0.041735,0.010144,0.088304,0.126208,0.261923,0.757107,0.011361


In [17]:
#creating a scaler function which can take a dataset and scale the column values 
#which are a continuous columns and have unique value greater then 10
def scaler(data):
    scaler = StandardScaler()
    for i in data.columns:
        if data[i].dtype!='object' and i!='index':
            j = data[i].nunique()
            if j>10:
                data[i] = pd.DataFrame(scaler.fit_transform(data[[i]].values.reshape(-1, 1)),columns=[i])
    return data

In [18]:
# we will use regression model to impute missing values. so scale all continuous variables preseted in dataframe
hf_train = scaler(hf_train)
hf_test = scaler(hf_test)

In [19]:
# copy testset for later use
re_test = hf_test.copy()

In [20]:
#creating a function which take two datasets,one is without missing value for fit in iterative_imputer and another dataset is with missing value which we want to fill.Return that dataset with imputed values.
def iterative_imputer(cleaned_data,data,cols):
    imputer = IterativeImputer(estimator=DecisionTreeRegressor(max_features='sqrt', random_state=0), max_iter=10, verbose=0, imputation_order='random', initial_strategy="most_frequent", random_state=0)
    imputer.fit(cleaned_data[cols])
    tmp = imputer.transform(data[cols])
    result = pd.DataFrame(tmp,columns = cols)
    data[cols] = result
    return data

In [21]:
#creating a dataset by selecting continuous cols in the dataframes 
ex_train = hf_train[['Transported','group','travelgroupos','travelgroup','cabin_1','Age','RoomService','FoodCourt','Spa','ShoppingMall','VRDeck','avgspent','transported_percentage','num_fam_mem','totalbill']]
ex_test = hf_test[['group','travelgroupos','travelgroup','cabin_1','Age','RoomService','FoodCourt','Spa','ShoppingMall','VRDeck','avgspent','transported_percentage','num_fam_mem','totalbill']]

In [22]:
cols = ['cabin_1','Age','RoomService','FoodCourt','Spa','ShoppingMall','VRDeck','avgspent','transported_percentage','num_fam_mem','totalbill']
#creating dataframe by concatenating train and test datasets with 0 missing value 
cleaned_data = pd.concat([ex_train.dropna(),ex_test.dropna()],axis=0)

ex_train = iterative_imputer(cleaned_data,ex_train,cols=cols)
ex_test = iterative_imputer(cleaned_data,ex_test,cols=cols)

after using IterativeImputation we get train and test dataset with no missing values.
we further use those dataset to fill missing values in previous train and test dataset by using regression imputation where we will use gradient boosting as regression method.

In [23]:
# creating a function which use iterativeimputationed dataset(no missing val) to predict missing values
def GradientBoosting_regression_imputation(data_to_be_impute,data_to_be_used,missing_var,linked_var):
    #taking 2 datasets as input 
    a = data_to_be_impute
    b = data_to_be_used
    #a_1 contains rows with missing values in target col and a_2 cotains rows without missing values in target col of target dataset
    a_1 = a[a[missing_var].isnull()]
    a_2 = a[~a[missing_var].isnull()]
    #b_1 is a subset of dataset b that includes only the rows for which the corresponding value in the target column of dataset a is missing
    #b_2 is a subset of dataset b that includes only the rows for which the corresponding value in the target column of dataset a is not missing
    b_1 = b[a[missing_var].isnull()]
    b_2 = b[~a[missing_var].isnull()]
    
    
    model = GradientBoostingRegressor()
    model.fit(b_2[linked_var],b_2[missing_var])#model fitted on imputed dataset 
    
    missing_val = model.predict(b_1[linked_var])
    a_1[missing_var] = missing_val
    
    data_to_be_impute = pd.concat([a_1,a_2],axis=0)
    return data_to_be_impute

In [24]:
#creating a function to impute categorical features which will use logistic regression for imputation
def logistic_regression_imputation(data_to_be_impute,data_to_be_used,missing_var,linked_var):
    #taking 2 datasets as input 
    a = data_to_be_impute
    b = data_to_be_used
    #a_1 contains rows with missing values in target col and a_2 cotains rows without missing values in target col of target dataset
    a_1 = a[a[missing_var].isnull()]
    a_2 = a[~a[missing_var].isnull()]
    #b_1 is a subset of dataset b that includes only the rows for which the corresponding value in the target column of dataset a is missing
    #b_2 is a subset of dataset b that includes only the rows for which the corresponding value in the target column of dataset a is not missing
    b_1 = b[a[missing_var].isnull()]
    b_2 = b[~a[missing_var].isnull()]
    
    model = LogisticRegression()
    model.fit(b_2[linked_var],b_2[missing_var])
    
    missing_val = model.predict(b_1[linked_var])
    a_1[missing_var] = missing_val
    
    data_to_be_impute = pd.concat([a_1,a_2],axis=0)
    return data_to_be_impute

In [25]:
# creating a function to impute categorical features which will use logistic regression for imputation after performing labelencoding on target feature
def categorical_regression_imputing(data,missing_var,linked_var):
    missing = data[data[missing_var].isnull()]
    without_missing = data[~data[missing_var].isnull()]
    le = LabelEncoder()
    without_missing[missing_var] = le.fit_transform(without_missing[missing_var])
    lr = LogisticRegression()
    lr.fit(without_missing[linked_var],without_missing[missing_var])
    missing_val = lr.predict(missing[linked_var])
    missing[missing_var] = missing_val
    data = pd.concat([missing,without_missing],axis=0)
    return data

In [26]:
hf_train.corr().T

,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,travelgroup,travelgroupos,cabin_1,totalbill,avgspent,transported_percentage,num_fam_mem,group,index
CryoSleep,1.000000,-0.074273,-0.253106,-0.210388,-0.212970,-0.202699,-0.196793,0.468645,-0.005879,0.064676,-0.037560,-0.385749,-0.009255,0.230689,-0.009337,0.082651,-0.006047
Age,-0.074273,1.000000,0.081597,0.132659,0.043975,0.127047,0.105447,-0.075026,-0.009439,-0.132756,-0.132255,0.189475,0.247943,0.093525,-0.133882,-0.179102,-0.009530
RoomService,-0.253106,0.081597,1.000000,-0.013658,0.057001,0.011243,-0.017582,-0.243211,0.002568,-0.024342,-0.007678,0.228603,0.028218,-0.129535,-0.029450,-0.043795,0.002671
FoodCourt,-0.210388,0.132659,-0.013658,1.000000,-0.012174,0.221847,0.229388,0.046837,-0.010860,0.023486,-0.183557,0.745826,0.353494,0.260452,-0.102376,0.031887,-0.010863
ShoppingMall,-0.212970,0.043975,0.057001,-0.012174,1.000000,0.016414,-0.004939,0.008410,0.015958,-0.040284,0.002708,0.218432,0.028043,-0.058171,-0.035127,-0.041727,0.016105
Spa,-0.202699,0.127047,0.011243,0.221847,0.016414,1.000000,0.155207,-0.219295,-0.006371,0.016871,-0.131631,0.596082,0.247401,0.167484,-0.068378,0.018564,-0.006223
VRDeck,-0.196793,0.105447,-0.017582,0.229388,-0.004939,0.155207,1.000000,-0.204962,0.015005,0.009995,-0.138942,0.572192,0.279455,0.200502,-0.075883,0.008679,0.014895
Transported,0.468645,-0.075026,-0.243211,0.046837,0.008410,-0.219295,-0.204962,1.000000,0.021491,0.066390,-0.045097,-0.197671,0.130525,0.214905,-0.033312,0.082644,0.021775
travelgroup,-0.005879,-0.009439,0.002568,-0.010860,0.015958,-0.006371,0.015005,0.021491,1.000000,0.011170,0.679723,-0.001793,0.004100,0.002985,0.004863,0.014753,0.999947
travelgroupos,0.064676,-0.132756,-0.024342,0.023486,-0.040284,0.016871,0.009995,0.066390,0.011170,1.000000,-0.041735,0.010144,0.088304,0.126208,0.261923,0.757107,0.011361


In [27]:
hf_train.isnull().sum()

HomePlanet                201
CryoSleep                 217
Destination               182
Age                       179
RoomService               158
FoodCourt                 170
ShoppingMall              185
Spa                       162
VRDeck                    163
Transported                 0
travelgroup                 0
travelgroupos               0
cabin_0                   199
cabin_1                   199
cabin_2                   199
totalbill                 908
avgspent                  199
transported_percentage    199
num_fam_mem               200
group                       0
index                       0
dtype: int64

In [28]:
ex_train

,Transported,group,travelgroupos,travelgroup,cabin_1,Age,RoomService,FoodCourt,Spa,ShoppingMall,VRDeck,avgspent,transported_percentage,num_fam_mem,totalbill
0,0,1,1,-1.734409,-1.172966,0.702095,-0.337216,-0.282663,-0.271609,-0.287546,-0.263545,2785.142490,0.734275,-1.531304,-0.521809
1,1,1,1,-1.734034,-1.172966,-0.333233,-0.168938,-0.277070,0.212456,-0.244923,-0.224968,893.478525,0.439871,-0.493862,-0.263119
2,0,2,1,-1.733660,-1.172966,2.013510,-0.270831,1.939449,5.649144,-0.287546,-0.220585,3230.875000,0.496094,0.197765,3.127616
3,0,2,2,-1.733660,-1.172966,0.287964,-0.337216,0.514588,2.663639,0.344977,-0.094334,3230.875000,0.496094,0.197765,1.297456
4,1,1,1,-1.733286,-1.171013,-0.885407,0.130566,-0.239165,0.226563,-0.030104,-0.261792,893.478525,0.439871,0.197765,-0.138343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0,1,1,1.738236,-0.981499,0.840138,-0.337216,3.954636,1.177058,-0.287546,-0.198666,3230.875000,0.496094,-0.839676,2.478431
8689,0,1,1,1.738984,1.755700,-0.747364,-0.337216,-0.282663,-0.271609,-0.287546,-0.263545,343.674873,0.516217,-1.185490,-0.521809
8690,1,1,1,1.739359,1.757654,-0.195189,-0.337216,-0.282663,-0.270727,2.904051,-0.263545,343.674873,0.516217,0.197765,0.136515
8691,0,2,1,1.739733,0.014912,0.218942,-0.337216,0.369182,0.039639,-0.287546,2.572713,1186.570776,0.357306,0.197765,1.108008


In [29]:
#selecting linked var by using correlation between missing var with other variables
hf_train = GradientBoosting_regression_imputation(hf_train,ex_train,missing_var='cabin_1',linked_var=['travelgroup','avgspent','num_fam_mem','VRDeck'])
hf_train = GradientBoosting_regression_imputation(hf_train,ex_train,missing_var='Age',linked_var=['avgspent','Spa','FoodCourt','VRDeck','group'])
hf_train = GradientBoosting_regression_imputation(hf_train,ex_train,missing_var='RoomService',linked_var=['totalbill','Age','avgspent','num_fam_mem','transported_percentage','Transported'])
hf_train = GradientBoosting_regression_imputation(hf_train,ex_train,missing_var='FoodCourt',linked_var=['totalbill','avgspent','VRDeck','Spa'])
hf_train = GradientBoosting_regression_imputation(hf_train,ex_train,missing_var='Spa',linked_var=['totalbill','FoodCourt','VRDeck','avgspent','Transported'])
hf_train = GradientBoosting_regression_imputation(hf_train,ex_train,missing_var='ShoppingMall',linked_var=['RoomService','totalbill','Age'])
hf_train = GradientBoosting_regression_imputation(hf_train,ex_train,missing_var='VRDeck',linked_var=['transported_percentage','totalbill','Transported','FoodCourt'])
hf_train = GradientBoosting_regression_imputation(hf_train,ex_train,missing_var='avgspent',linked_var=['cabin_1','Age','FoodCourt','num_fam_mem'])
hf_train = GradientBoosting_regression_imputation(hf_train,ex_train,missing_var='transported_percentage',linked_var=['cabin_1','totalbill','avgspent','totalbill'])
hf_train = GradientBoosting_regression_imputation(hf_train,ex_train,missing_var='num_fam_mem',linked_var=['Age','group','avgspent','RoomService'])
#hf_train = GradientBoosting_regression_imputation(hf_train,ex_train,missing_var='totalbill',linked_var=['avgspent','FoodCourt','RoomService','Spa','ShoppingMall','VRDeck'])

hf_test = GradientBoosting_regression_imputation(hf_test,ex_test,missing_var='cabin_1',linked_var=['travelgroup','avgspent','num_fam_mem','VRDeck'])
hf_test = GradientBoosting_regression_imputation(hf_test,ex_test,missing_var='Age',linked_var=['avgspent','Spa','FoodCourt','VRDeck','group'])
hf_test = GradientBoosting_regression_imputation(hf_test,ex_test,missing_var='RoomService',linked_var=['totalbill','transported_percentage','Age','avgspent','num_fam_mem'])
hf_test = GradientBoosting_regression_imputation(hf_test,ex_test,missing_var='FoodCourt',linked_var=['totalbill','avgspent','VRDeck','Spa'])
hf_test = GradientBoosting_regression_imputation(hf_test,ex_test,missing_var='Spa',linked_var=['totalbill','FoodCourt','VRDeck','avgspent'])
hf_test = GradientBoosting_regression_imputation(hf_test,ex_test,missing_var='ShoppingMall',linked_var=['RoomService','totalbill','Age','FoodCourt'])
hf_test = GradientBoosting_regression_imputation(hf_test,ex_test,missing_var='VRDeck',linked_var=['transported_percentage','totalbill','FoodCourt'])
hf_test = GradientBoosting_regression_imputation(hf_test,ex_test,missing_var='avgspent',linked_var=['cabin_1','Age','FoodCourt','num_fam_mem'])
hf_test = GradientBoosting_regression_imputation(hf_test,ex_test,missing_var='transported_percentage',linked_var=['cabin_1','totalbill','avgspent'])
hf_test = GradientBoosting_regression_imputation(hf_test,ex_test,missing_var='num_fam_mem',linked_var=['Age','group','avgspent','RoomService'])
#hf_test = GradientBoosting_regression_imputation(hf_test,ex_test,missing_var='totalbill',linked_var=['avgspent','FoodCourt','RoomService','Spa','ShoppingMall','VRDeck'])

In [30]:
hf_train = categorical_regression_imputing(hf_train,'HomePlanet',['avgspent','Age','num_fam_mem','RoomService'])
hf_train = categorical_regression_imputing(hf_train,'Destination',['transported_percentage','FoodCourt','Transported','VRDeck'])
hf_train = categorical_regression_imputing(hf_train,'cabin_0',['HomePlanet','cabin_1','avgspent','transported_percentage','FoodCourt'])
hf_train = categorical_regression_imputing(hf_train,'cabin_2',['Transported','ShoppingMall','VRDeck','Spa','Age'])
hf_train = categorical_regression_imputing(hf_train,'CryoSleep',['Transported','RoomService','FoodCourt','ShoppingMall','Age'])

hf_test = categorical_regression_imputing(hf_test,'HomePlanet',['avgspent','Age','num_fam_mem','RoomService'])
hf_test = categorical_regression_imputing(hf_test,'Destination',['transported_percentage','FoodCourt','VRDeck'])
hf_test = categorical_regression_imputing(hf_test,'cabin_0',['HomePlanet','cabin_1','avgspent','transported_percentage','FoodCourt'])
hf_test = categorical_regression_imputing(hf_test,'cabin_2',['ShoppingMall','VRDeck','Spa','Age'])
hf_test = categorical_regression_imputing(hf_test,'CryoSleep',['RoomService','FoodCourt','ShoppingMall','Spa','Age'])

In [31]:
# further updating totalbill col to fill missing values in this col
hf_train['totalbill'] = hf_train['RoomService'] + hf_train['FoodCourt'] + hf_train['Spa'] + hf_train['ShoppingMall'] + hf_train['VRDeck']

hf_test['totalbill'] = hf_test['RoomService'] + hf_test['FoodCourt'] + hf_test['Spa'] + hf_test['ShoppingMall'] + hf_test['VRDeck']

In [32]:
hf_train.isnull().sum()

HomePlanet                0
CryoSleep                 0
Destination               0
Age                       0
RoomService               0
FoodCourt                 0
ShoppingMall              0
Spa                       0
VRDeck                    0
Transported               0
travelgroup               0
travelgroupos             0
cabin_0                   0
cabin_1                   0
cabin_2                   0
totalbill                 0
avgspent                  0
transported_percentage    0
num_fam_mem               0
group                     0
index                     0
dtype: int64

In [33]:
hf_test.isnull().sum()

HomePlanet                0
CryoSleep                 0
Destination               0
Age                       0
RoomService               0
FoodCourt                 0
ShoppingMall              0
Spa                       0
VRDeck                    0
travelgroup               0
travelgroupos             0
cabin_0                   0
cabin_1                   0
cabin_2                   0
totalbill                 0
avgspent                  0
transported_percentage    0
num_fam_mem               0
group                     0
index                     0
dtype: int64

As so far we get a clean train and test datasets. This train dataset we will use for further process and traing. But to update missing values more accurately in test dataset we will concat, (updated train and updated test) and also concat (upadated train and re_test dateset) which we copied previously after some feature manipulation which contain missing values. And pass it to GradientBoosting_regression_imputation and categorical_regression_imputing for imputaion.

In [34]:
#though test dataset don't has dependent variable, we drop the col 
reusable_train = hf_train.drop(['Transported'],axis=1)

In [35]:
#defining a function which perform labelencoding on selected cols and return the dataset
def label_encoding(data,cols):
    le = LabelEncoder()
    for i in cols:
        missing = data[data[i].isnull()]
        without_missing = data[~data[i].isnull()]
        without_missing[i] = le.fit_transform(without_missing[i])
        data = pd.concat([missing,without_missing],axis=0)
    return data
cols = ['HomePlanet','Destination','cabin_0','cabin_2','CryoSleep']
re_test = label_encoding(re_test,cols)

In [36]:
#(8693rows + 4277rows) = 12970rows
new_test_with_missing = pd.concat([re_test,reusable_train],axis=0).reset_index()
new_test_without_missing = pd.concat([hf_test,reusable_train],axis=0).reset_index()

In [37]:
new_test_with_missing

,level_0,HomePlanet,CryoSleep,Destination,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,...,travelgroupos,cabin_0,cabin_1,cabin_2,totalbill,avgspent,transported_percentage,num_fam_mem,group,index
0,99,0,NaN,2,-0.469632,-0.354425,-0.286027,-0.312997,-0.269032,-0.247740,...,1,NaN,NaN,NaN,-0.509762,NaN,NaN,2.045231,1,99
1,3389,0,NaN,2,0.729462,0.961810,-0.286027,-0.312997,-0.184704,-0.247740,...,1,NaN,NaN,NaN,-0.196843,NaN,NaN,1.123781,2,3389
2,4209,1,NaN,2,0.870531,-0.354425,1.023329,-0.312997,0.648707,0.449316,...,1,NaN,NaN,NaN,0.864961,NaN,NaN,-1.179844,1,4209
3,509,0,NaN,NaN,-0.892841,-0.354425,-0.286027,-0.312997,-0.269032,-0.247740,...,1,6,-0.866524,0,-0.509762,343.674873,0.516217,-1.179844,1,509
4,1559,1,NaN,NaN,-0.116957,-0.354425,0.778234,-0.312997,-0.221485,0.678452,...,1,4,-0.744172,1,0.490519,1186.570776,0.357306,-1.179844,1,1559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12965,8688,1,0.0,0,0.840138,-0.337216,3.954636,-0.287546,1.177058,-0.198666,...,1,0,-0.981499,0,4.308267,3230.875000,0.496094,-0.839676,1,12965
12966,8689,0,1.0,1,-0.747364,-0.337216,-0.282663,-0.287546,-0.271609,-0.263545,...,1,6,1.755700,1,-1.442578,343.674873,0.516217,-1.185490,1,12966
12967,8690,0,0.0,2,-0.195189,-0.337216,-0.282663,2.904051,-0.270727,-0.263545,...,1,6,1.757654,1,1.749901,343.674873,0.516217,0.197765,1,12967
12968,8691,1,0.0,0,0.218942,-0.337216,0.369182,-0.287546,0.039639,2.572713,...,1,4,0.014912,1,2.356772,1186.570776,0.357306,0.197765,2,12968


In [38]:
# performing same oparetions to get more accurate test data
new_test_with_missing = GradientBoosting_regression_imputation(new_test_with_missing,new_test_without_missing,missing_var='cabin_1',linked_var=['travelgroup','avgspent','num_fam_mem','VRDeck'])
new_test_with_missing = GradientBoosting_regression_imputation(new_test_with_missing,new_test_without_missing,missing_var='Age',linked_var=['avgspent','Spa','FoodCourt','VRDeck','group'])
new_test_with_missing = GradientBoosting_regression_imputation(new_test_with_missing,new_test_without_missing,missing_var='RoomService',linked_var=['totalbill','transported_percentage'])
new_test_with_missing = GradientBoosting_regression_imputation(new_test_with_missing,new_test_without_missing,missing_var='FoodCourt',linked_var=['totalbill','avgspent','VRDeck','Spa'])
new_test_with_missing = GradientBoosting_regression_imputation(new_test_with_missing,new_test_without_missing,missing_var='Spa',linked_var=['totalbill','FoodCourt','VRDeck','avgspent'])
new_test_with_missing = GradientBoosting_regression_imputation(new_test_with_missing,new_test_without_missing,missing_var='ShoppingMall',linked_var=['RoomService','totalbill','Age','FoodCourt'])
new_test_with_missing = GradientBoosting_regression_imputation(new_test_with_missing,new_test_without_missing,missing_var='VRDeck',linked_var=['transported_percentage','totalbill','FoodCourt'])
new_test_with_missing = GradientBoosting_regression_imputation(new_test_with_missing,new_test_without_missing,missing_var='avgspent',linked_var=['cabin_1','Age','FoodCourt','num_fam_mem'])
new_test_with_missing = GradientBoosting_regression_imputation(new_test_with_missing,new_test_without_missing,missing_var='transported_percentage',linked_var=['cabin_1','totalbill','avgspent'])
new_test_with_missing = GradientBoosting_regression_imputation(new_test_with_missing,new_test_without_missing,missing_var='num_fam_mem',linked_var=['Age','group','avgspent'])
#new_test_with_missing = GradientBoosting_regression_imputation(new_test_with_missing,new_test_without_missing,missing_var='totalbill',linked_var=['avgspent','FoodCourt','RoomService','Spa','ShoppingMall','VRDeck'])

In [39]:
new_test_with_missing = logistic_regression_imputation(new_test_with_missing,new_test_without_missing,'HomePlanet',['avgspent','Age','num_fam_mem','RoomService'])
new_test_with_missing = logistic_regression_imputation(new_test_with_missing,new_test_without_missing,'Destination',['transported_percentage','FoodCourt','VRDeck'])
new_test_with_missing = logistic_regression_imputation(new_test_with_missing,new_test_without_missing,'cabin_0',['HomePlanet','cabin_1','avgspent','transported_percentage','FoodCourt'])
new_test_with_missing = logistic_regression_imputation(new_test_with_missing,new_test_without_missing,'cabin_2',['ShoppingMall','VRDeck','Spa','Age'])
new_test_with_missing = logistic_regression_imputation(new_test_with_missing,new_test_without_missing,'CryoSleep',['RoomService','FoodCourt','ShoppingMall','Age'])

In [40]:
new_test_with_missing['totalbill'] = new_test_with_missing['RoomService'] + new_test_with_missing['FoodCourt'] + new_test_with_missing['Spa'] + new_test_with_missing['ShoppingMall'] + new_test_with_missing['VRDeck']

In [41]:
new_test_with_missing = new_test_with_missing.sort_values('index')

In [42]:
test = new_test_with_missing[new_test_with_missing['index'] < len(hf_test)]#extracting test dataset by using index column we creat earlier.
test = test.sort_values('index').drop(['index'],axis=1)#after sorting rows by index col values, dropping index
hf_train = hf_train.drop(['index'],axis=1)


In [43]:
test = test.drop(['level_0'],axis=1)


In [44]:
def one_hot_encoding(df,cols):
    for i in cols:
        p = pd.get_dummies(df[i],prefix=i)
        df = pd.concat([df,p],axis=1)
    df = df.drop(columns=cols)
    return df
cols = ['HomePlanet','Destination','cabin_0','cabin_2','CryoSleep']
train = one_hot_encoding(hf_train,cols)# performing one hot encoding in categorical cols
test = one_hot_encoding(test,cols)


In [45]:
test['CryoSleep_0'] = test['CryoSleep_0.0']
test['CryoSleep_1'] = test['CryoSleep_1.0']

In [46]:
x = train.drop(['Transported','travelgroup'],axis=1)
y = train['Transported'].astype('int')
test = test.drop(['travelgroup'],axis=1)

In [47]:
x

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,travelgroupos,cabin_1,totalbill,avgspent,...,cabin_0_2,cabin_0_3,cabin_0_4,cabin_0_5,cabin_0_6,cabin_0_7,cabin_2_0,cabin_2_1,CryoSleep_0,CryoSleep_1
6057,-1.851713,-0.337216,-0.282663,-0.287546,-0.271609,-0.263545,2,1.320383,-1.442578,834.419661,...,0,0,0,1,0,0,0,1,0,1
314,0.495029,0.465578,-0.282663,2.892117,-0.271609,-0.263545,2,-1.023590,2.539878,654.037401,...,0,0,1,0,0,0,0,1,1,0
691,1.737422,-0.337216,-0.282663,-0.287546,-0.271609,-0.263545,1,-0.872878,-1.442578,2661.474740,...,1,0,0,0,0,0,0,1,0,1
1049,0.495029,-0.337216,1.588361,-0.287546,-0.271609,-0.177624,2,-0.749934,0.514366,654.037401,...,0,0,0,0,0,0,0,1,1,0
1219,-0.678342,-0.232235,-0.282663,-0.287546,0.362348,-0.263545,1,-0.692676,-0.703640,463.650054,...,0,0,0,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.840138,-0.337216,3.954636,-0.287546,1.177058,-0.198666,1,-0.981499,4.308267,3230.875000,...,0,0,0,0,0,0,1,0,1,0
8689,-0.747364,-0.337216,-0.282663,-0.287546,-0.271609,-0.263545,1,1.755700,-1.442578,343.674873,...,0,0,0,0,1,0,0,1,0,1
8690,-0.195189,-0.337216,-0.282663,2.904051,-0.270727,-0.263545,1,1.757654,1.749901,343.674873,...,0,0,0,0,1,0,0,1,1,0
8691,0.218942,-0.337216,0.369182,-0.287546,0.039639,2.572713,1,0.014912,2.356772,1186.570776,...,0,0,1,0,0,0,0,1,1,0


In [48]:
#arranging test cols as train cols
col_order = x.columns.tolist()
test = test[col_order]

In [49]:
# convert to numpy because gokinjo expects np arrays
X = x.to_numpy()
y = y.to_numpy()
test_x = test.to_numpy()

In [50]:
# extracting new features using knn kfold extract
KNN_feat_train = knn_kfold_extract(X, y, k=2, normalize='standard')
KNN_feat_test = knn_extract(X, y, test_x, k=2, normalize='standard')
KNN_feat_train

array([[2.33216896, 5.64836396, 0.95393461, 3.072082  ],
       [2.19667693, 4.4330299 , 2.13600385, 4.40285105],
       [4.33805261, 8.93336364, 2.47049294, 4.96286365],
       ...,
       [2.32913314, 5.1803756 , 2.18191465, 4.45671738],
       [1.7702045 , 3.65626631, 3.82331641, 8.21020844],
       [2.36735442, 5.10951138, 1.37958891, 3.52134327]])

In [51]:
# add KNN feature to normal features
X, test_x = np.append(X, KNN_feat_train, axis=1), np.append(test_x, KNN_feat_test, axis=1) 
print("Train set, shape: ", np.shape(X))
print("Test set, shape: ", np.shape(test_x))

Train set, shape:  (8693, 35)
Test set, shape:  (4277, 35)


In [52]:
b = list(x.columns)
x = pd.DataFrame(X)
x.columns = b+['k1','k2','k3','k4']
test = pd.DataFrame(test_x)
test.columns= b+['k1','k2','k3','k4']

In [53]:
x

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,travelgroupos,cabin_1,totalbill,avgspent,...,cabin_0_6,cabin_0_7,cabin_2_0,cabin_2_1,CryoSleep_0,CryoSleep_1,k1,k2,k3,k4
0,-1.851713,-0.337216,-0.282663,-0.287546,-0.271609,-0.263545,2.0,1.320383,-1.442578,834.419661,...,0.0,0.0,0.0,1.0,0.0,1.0,2.332169,5.648364,0.953935,3.072082
1,0.495029,0.465578,-0.282663,2.892117,-0.271609,-0.263545,2.0,-1.023590,2.539878,654.037401,...,0.0,0.0,0.0,1.0,1.0,0.0,2.196677,4.433030,2.136004,4.402851
2,1.737422,-0.337216,-0.282663,-0.287546,-0.271609,-0.263545,1.0,-0.872878,-1.442578,2661.474740,...,0.0,0.0,0.0,1.0,0.0,1.0,4.338053,8.933364,2.470493,4.962864
3,0.495029,-0.337216,1.588361,-0.287546,-0.271609,-0.177624,2.0,-0.749934,0.514366,654.037401,...,0.0,0.0,0.0,1.0,1.0,0.0,4.194778,8.674852,4.428425,8.892624
4,-0.678342,-0.232235,-0.282663,-0.287546,0.362348,-0.263545,1.0,-0.692676,-0.703640,463.650054,...,1.0,0.0,1.0,0.0,1.0,0.0,0.787667,1.685767,1.046455,2.143196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.840138,-0.337216,3.954636,-0.287546,1.177058,-0.198666,1.0,-0.981499,4.308267,3230.875000,...,0.0,0.0,1.0,0.0,1.0,0.0,2.392416,5.368767,2.260491,4.783899
8689,-0.747364,-0.337216,-0.282663,-0.287546,-0.271609,-0.263545,1.0,1.755700,-1.442578,343.674873,...,1.0,0.0,0.0,1.0,0.0,1.0,1.223553,2.844764,0.493612,1.134901
8690,-0.195189,-0.337216,-0.282663,2.904051,-0.270727,-0.263545,1.0,1.757654,1.749901,343.674873,...,1.0,0.0,0.0,1.0,1.0,0.0,2.329133,5.180376,2.181915,4.456717
8691,0.218942,-0.337216,0.369182,-0.287546,0.039639,2.572713,1.0,0.014912,2.356772,1186.570776,...,0.0,0.0,0.0,1.0,1.0,0.0,1.770205,3.656266,3.823316,8.210208


In [54]:
#pt=PowerTransformer()
#x= pd.DataFrame(pt.fit_transform(x),columns=x.columns)
#test = pd.DataFrame(pt.fit_transform(test),columns=test.columns)

In [55]:
def outlier(data):
    for i in data.columns:
        if data[i].dtype=='int64' or 'float64':
            p25 = data[i].quantile(0.25)
            p75 = data[i].quantile(0.75)
            iqr = p75-p25
            upper = p75 + 1.5*iqr
            lower = p25 - 1.5*iqr
            data[i] = np.where(
            data[i]>upper,
            upper,
            np.where(
            data[i]<lower,
            lower,
            data[i]))
    return data
#x=outlier(x)
#test=outlier(test)

In [56]:
#scaler = StandardScaler()
#x = scaler.fit_transform(x)
#test = scaler.fit_transform(test)

In [57]:
#splitting train data for traing and validation
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [58]:
#logistic regression
lr = LogisticRegression()
params = {
    'penalty':['l1','l2'],
    'C':[0.1],
    'solver':['newton-cg'],
    'max_iter':[100,150,200,300],
    'class_weight':[None]
}
cv = KFold(n_splits=5, shuffle=True, random_state=42)
grid = GridSearchCV(estimator = lr,param_grid = params,cv=cv)
grid.fit(x_train,y_train)
model_0 = grid.best_estimator_
model_0.fit(x_train,y_train)
pred = model_0.predict(x_test)
print(accuracy_score(y_test,pred))
print(r2_score(y_test.astype(int),pred.astype(int)))

0.8039102932719954
0.2156097883597885


In [59]:
#param_grid = {
#    'learning_rate': [0.1, 0.01, 0.001],
#    'n_estimators': [50, 100, 200],
#    'max_depth': [3, 5, 7],
#   'min_samples_split': [2, 5, 10]
#}
#gbr = GradientBoostingRegressor()
#grid_search = GridSearchCV(
#    gbr, 
#    param_grid=param_grid, 
#    cv=5
#)
#grid_search.fit(x_train,y_train)
#model_1 = grid_search.best_estimator_
#model_1.fit(x_train,y_train)
#pred = model_1.predict(x_test)
#pred = np.where(pred>=0.5,1,0)
#print(r2_score(y_test.astype(int),pred.astype(int)))
#print(mean_squared_error(y_test.astype(int),pred.astype(int)))
#print(accuracy_score(y_test,pred.astype('int')))


In [60]:
#GradientBoostingRegressor
model_2 = GradientBoostingRegressor()
model_2.fit(x_train,y_train)
predict = model_2.predict(x_test)
pred = np.where(predict>=0.5,1,0)
print(r2_score(y_test.astype(int),pred.astype(int)))
print(mean_squared_error(y_test.astype(int),pred.astype(int)))
print(accuracy_score(y_test,pred.astype('int')))

0.2501137566137568
0.18746405980448533
0.8125359401955147


In [61]:
p=pd.DataFrame({'f':x_train.columns,'imp':model_2.feature_importances_})
p.sort_values('imp')
q = p.nlargest(20, 'imp')['f'].tolist()
print('top 20 features :',q)

top 20 features : ['CryoSleep_1', 'CryoSleep_0', 'FoodCourt', 'totalbill', 'Spa', 'VRDeck', 'k2', 'RoomService', 'HomePlanet_0', 'cabin_1', 'ShoppingMall', 'k3', 'Age', 'transported_percentage', 'k1', 'k4', 'cabin_2_0', 'cabin_2_1', 'avgspent', 'group']


In [62]:
import tensorflow
from tensorflow import keras
from keras import Sequential,regularizers
from keras.layers import Dense,BatchNormalizationV2,Dropout
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,LearningRateScheduler
import math

In [63]:
NN_model_1 = Sequential()

In [64]:
# creating ANN architecture
NN_model_1.add(Dense(1024,input_dim=x_train.shape[1],kernel_regularizer=regularizers.L2(65e-6),
                     kernel_initializer='he_normal',activation='elu'))
NN_model_1.add(BatchNormalizationV2())
NN_model_1.add(Dropout(0.2))
NN_model_1.add(Dense(1024,kernel_regularizer=regularizers.L2(0.001),
                     kernel_initializer='he_normal',activation='elu'))
NN_model_1.add(BatchNormalizationV2())
NN_model_1.add(Dropout(0.2))
NN_model_1.add(Dense(512,kernel_regularizer=regularizers.L2(0.001),
                     kernel_initializer='he_normal',activation='elu'))
NN_model_1.add(BatchNormalizationV2())
NN_model_1.add(Dropout(0.2))
NN_model_1.add(Dense(256,kernel_regularizer=regularizers.L2(0.001),
                     kernel_initializer='he_normal',activation='elu'))
NN_model_1.add(BatchNormalizationV2())
NN_model_1.add(Dropout(0.2))
NN_model_1.add(Dense(64,kernel_regularizer=regularizers.L2(0.001),
                     kernel_initializer='he_normal',activation='elu'))
NN_model_1.add(BatchNormalizationV2())
NN_model_1.add(Dropout(0.1))
NN_model_1.add(Dense(32,kernel_regularizer=regularizers.L2(0.001),
                     kernel_initializer='he_normal',activation='elu'))
NN_model_1.add(BatchNormalizationV2())
NN_model_1.add(Dropout(0.1))
NN_model_1.add(Dense(16,kernel_regularizer=regularizers.L2(0.001),
                     kernel_initializer='he_normal',activation='elu'))
NN_model_1.add(BatchNormalizationV2())
NN_model_1.add(Dropout(0.1))

NN_model_1.add(Dense(8,kernel_regularizer=regularizers.L2(0.001),
                     kernel_initializer='he_normal',activation='elu'))
NN_model_1.add(BatchNormalizationV2())
NN_model_1.add(Dropout(0.1))

NN_model_1.add(Dense(1,activation='sigmoid'))

In [65]:
def scheduler(epochs,lr):
    if epochs<10:
        return lr
    else:
        return lr*math.exp(-1)

In [66]:
NN_model_1.compile(optimizer='RMSprop',loss='BinaryCrossentropy',metrics=['accuracy'])
lr_reducer= ReduceLROnPlateau(monitor='val_loss',patience=5,mode='auto',min_lr=0.0005)
early_stopping = EarlyStopping(monitor='val_loss',patience = 8,restore_best_weights=True,)
lr_scheduler = LearningRateScheduler(scheduler)

In [67]:
history = NN_model_1.fit(x_train,y_train,batch_size=128,callbacks=[lr_reducer,early_stopping],
                        validation_data=(x_test,y_test),epochs=70)

Epoch 1/70
55/55 [==============================] - 7s 53ms/step - loss: 4.0868 - accuracy: 0.6619 - val_loss: 4.8751 - val_accuracy: 0.5256 - lr: 0.0010
Epoch 2/70
55/55 [==============================] - 2s 38ms/step - loss: 3.0170 - accuracy: 0.7361 - val_loss: 3.1455 - val_accuracy: 0.6555 - lr: 0.0010
Epoch 3/70
55/55 [==============================] - 2s 38ms/step - loss: 2.2267 - accuracy: 0.7570 - val_loss: 2.4517 - val_accuracy: 0.5963 - lr: 0.0010
Epoch 4/70
55/55 [==============================] - 2s 38ms/step - loss: 1.6934 - accuracy: 0.7561 - val_loss: 2.2449 - val_accuracy: 0.5739 - lr: 0.0010
Epoch 5/70
55/55 [==============================] - 2s 40ms/step - loss: 1.3144 - accuracy: 0.7632 - val_loss: 1.1940 - val_accuracy: 0.7556 - lr: 0.0010
Epoch 6/70
55/55 [==============================] - 2s 38ms/step - loss: 1.0803 - accuracy: 0.7623 - val_loss: 1.2851 - val_accuracy: 0.5900 - lr: 0.0010
Epoch 7/70
55/55 [==============================] - 2s 38ms/step - loss: 0.9

In [68]:
#creating submission file
pred_value = model_2.predict(test)
pred_value = np.where(pred_value<0.50,0,1)
pred_value = pred_value.ravel()
pred_value = pred_value.astype('bool')
PassengerId = df_test['PassengerId']
data = pd.DataFrame({'PassengerId':PassengerId,'Transported':pred_value})
data.to_csv("submission.csv", index = False)